In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
scraper = cloudscraper.create_scraper()
headers = {

'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

}
dados_rj = ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina=1&transacao=aluguel'] + \
             ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina={}&transacao=aluguel'.format(i) for i in range(2, 323)]



In [5]:
data = []
for dados in dados_rj:
    response = scraper.get(dados, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    pagina_atual = soup.find_all('li', {'data-cy' : 'rp-property-cd'})
    
    for item in pagina_atual:
        local_element = item.find('h2', {'data-cy': 'rp-cardProperty-location-txt'})
        local = local_element.text.strip() if local_element else 'NA'

        area_element = item.find('li', {'data-cy': 'rp-cardProperty-propertyArea-txt'})
        area = area_element.text.strip() if area_element else 'NA'
        
        quartos_element = item.find('li', {'data-cy': 'rp-cardProperty-bedroomQuantity-txt'})
        quartos = quartos_element.text.strip() if quartos_element else 'NA'
        
        banheiros_element = item.find('li', {'data-cy': 'rp-cardProperty-bathroomQuantity-txt'})
        banheiros = banheiros_element.text.strip() if banheiros_element else 'NA'
        
        vagas_element = item.find('li', {'data-cy': 'rp-cardProperty-parkingSpacesQuantity-txt'})
        vagas = vagas_element.text.strip() if vagas_element else 'NA' 

        preco_element = item.find('div', {'data-cy' : 'rp-cardProperty-price-txt'})
        preco = preco_element.text.strip() if preco_element else 'NA'
        
        link_element = item.find('a', href=True)
        link = None
        if link_element and link_element['href'].startswith('https://www.zapimoveis.com.br/imovel/'):
            link = link_element['href']

        data.append({
            'Local': local, 
            'Preço': preco,
            'Área': area,
            'Quartos': quartos,
            'Banheiros': banheiros,
            'Vagas': vagas,
            'Link': link
        })
    
    time.sleep(1) 

In [6]:
database = pd.DataFrame(data)
database.to_csv('imoveis_rio.csv', index = False)